# **Vector Space Model**

Vektor Space Model (VSM) adalah cara untuk mewakili teks seperti dokumen dalam bentuk angka-angka. Setiap dokumen dianggap sebagai kumpulan kata, dan setiap kata diubah menjadi angka berdasarkan seberapa sering kata itu muncul. Dokumen kemudian digambarkan sebagai vektor, atau garis dengan arah tertentu, dalam ruang yang memiliki banyak dimensi (setiap kata adalah satu dimensi). Ketika kita ingin mencari dokumen yang mirip dengan pertanyaan atau teks tertentu, kita membandingkan vektor-vektor ini dan melihat seberapa dekat mereka satu sama lain. Semakin dekat vektor-vektor tersebut, semakin mirip dokumen-dokumennya.

## **Library**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/PPW/tugas/

/content/drive/My Drive/PPW/tugas


In [3]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.8 MB/s eta 0:00:00


In [18]:
import pandas as pd

import re
import string

#stopwords
import nltk
from nltk.corpus import stopwords

#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#spit data
from sklearn.model_selection import train_test_split

#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

#split data
from sklearn.model_selection import train_test_split

import joblib

from sklearn.metrics.pairwise import cosine_similarity

import pickle

## **Load Data**

In [19]:
df = pd.read_csv("Tugas-Crawling-Data-Berita-2-kategori.csv")
df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik
...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup


## **Preprocessing**

Pre-processing adalah tahap awal untuk membersihkan dan menyiapkan data dari internet agar mudah dipahami dan digunakan. Proses ini meliputi penghapusan data yang tidak relevan, format ulang data, normalisasi (menyatukan data dalam bentuk yang konsisten), dan penghilangan duplikasi. Tujuannya adalah untuk meningkatkan kualitas data sehingga analisis yang dilakukan lebih akurat dan efisien.

Misalnya, dalam menganalisis teks dari sebuah situs web, pre-processing bisa melibatkan penghapusan tag HTML, stop words (kata-kata umum yang tidak penting, seperti "dan" atau "yang"), dan pengubahan kata ke dalam bentuk dasar (stemming).

### **Mengubah kolom kategori menjadi bentuk biner**

In [20]:
df['Kategori'] = df['Kategori Berita'].map({'Politik': 0, 'Gaya Hidup': 1})
df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0
...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1


langkah-langkah dalam pre-processing:

### **1. Cleansing**



proses pembersihan data dari elemen-elemen yang tidak relevan atau mengganggu, cleansing menghapus elemen-elemen yang tidak mempengaruhi analisis (seperti sentimen).

berikut code untuk proses Cleansing:




In [30]:
def remove_symbols(data_berita):
  data_berita = re.sub(r'[^a-zA-Z0-9\s]', '', data_berita) #menghapus karakter
  return data_berita

df['cleansing'] = df['Isi Berita'].apply(lambda x: remove_symbols(x))

df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori,cleansing,case_folding
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0,TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...
...,...,...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1,TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1,TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1,TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...


,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori,cleansing,case_folding
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0,TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...
...,...,...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1,TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1,TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1,TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...


### **2. Case Folding**

Case folding adalah proses dalam pemrosesan teks yang melibatkan konversi semua karakter dalam teks menjadi bentuk huruf kecil (lowercase). Tujuan utama dari case folding adalah untuk mengurangi variasi yang tidak perlu dalam teks yang bisa disebabkan oleh perbedaan huruf besar dan kecil.

Misalnya, "Apple" dan "apple" dianggap sebagai kata yang sama setelah case folding.

berikut code untuk proses Case Folding:


In [31]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['cleansing'].apply(case_folding)

df[['Isi Berita', 'cleansing', 'case_folding']]

,Isi Berita,cleansing,case_folding
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...
...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...


### **3. Tokenization**



Tokenization adalah proses membagi teks menjadi unit-unit kata yang lebih kecil, yang disebut token. Dalam metode ini, setiap kata dalam sebuah teks diidentifikasi dan dipisahkan dari kata-kata lainnya.

Misalnya, kalimat "kuncoro pergi ke kantor" di-tokenize menjadi ["kuncoro", "pergi", "ke", "kantor"]. Proses ini memungkinkan analisis yang lebih terfokus pada kata-kata individu.

berikut code untuk proses Tokenization:


In [32]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df[['Isi Berita', 'cleansing', 'case_folding', 'tokenize']]

,Isi Berita,cleansing,case_folding,tokenize
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi..."
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan..."
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ..."
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb..."
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,..."
...,...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg..."
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...,"[timesindonesia, malang, pada, minggu, 192024,..."
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,..."
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic..."


### **4. Stop Words**




Stop word removal adalah proses menghilangkan kata-kata umum yang sering muncul dalam teks tetapi tidak memiliki makna yang signifikan dalam sebuah dokumen. Kata-kata seperti "dan," "atau," "adalah," dan "di" merupakan contoh stop word dalam bahasa Indonesia. Karena kata-kata ini tidak menambah informasi penting dalam konteks analisis teks atau pencarian informasi, mereka sering diabaikan agar fokus bisa tertuju pada kata-kata yang lebih relevan.

berikut code untuk proses Stop Words:

In [33]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

df[['Isi Berita', 'cleansing', 'case_folding', 'tokenize', 'stopword_removal']]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Isi Berita,cleansing,case_folding,tokenize,stopword_removal
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan...",timesindonesia malang cawabup malang lathifah ...
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ...",timesindonesia bantul rois syuriah pcnu bantul...
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb...",timesindonesia mojokerto ribuan kader pkb kabu...
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,...",timesindonesia jakarta partai kebangkitan bang...
...,...,...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg...",timesindonesia malang azquira scarf menggelar ...
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...,"[timesindonesia, malang, pada, minggu, 192024,...",timesindonesia malang minggu 192024 komunitas ...
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,...",timesindonesia surabaya memilih wewangian peng...
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic...",timesindonesia magelang mencicipi kenikmatan s...


### **Stemming**

Stemming adalah proses mengubah kata-kata yang memiliki berbagai bentuk turunan menjadi bentuk dasarnya. Tujuan utama stemming adalah untuk menyederhanakan teks dengan mengonversi kata-kata seperti "berlari", "pelari", dan "berlarian" menjadi "lari". Dengan demikian, komputer dapat memahami bahwa semua bentuk kata tersebut memiliki makna yang sama.

berikut code untuk stemming:

In [34]:
# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming(text):
    return [stemmer.stem(word) for word in text]

# Terapkan stemming setelah stopwords removal
df['stemming'] = df['stopword_removal'].apply(lambda x: ' '.join(stemming(x.split())))

df[['Isi Berita', 'cleansing', 'case_folding', 'tokenize', 'stopword_removal', 'stemming']]

,Isi Berita,cleansing,case_folding,tokenize,stopword_removal,stemming
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...,timesindonesia jakarta pimpin presiden ri prab...
1,"TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan...",timesindonesia malang cawabup malang lathifah ...,timesindonesia malang cawabup malang lathifah ...
2,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ...",timesindonesia bantul rois syuriah pcnu bantul...,timesindonesia bantul rois syuriah pcnu bantul...
3,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb...",timesindonesia mojokerto ribuan kader pkb kabu...,timesindonesia mojokerto ribu kader pkb kabupa...
4,"TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,...",timesindonesia jakarta partai kebangkitan bang...,timesindonesia jakarta partai bangkit bangsa p...
...,...,...,...,...,...,...
95,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg...",timesindonesia malang azquira scarf menggelar ...,timesindonesia malang azquira scarf gelar acar...
96,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",TIMESINDONESIA MALANG Pada Minggu 192024 komu...,timesindonesia malang pada minggu 192024 komu...,"[timesindonesia, malang, pada, minggu, 192024,...",timesindonesia malang minggu 192024 komunitas ...,timesindonesia malang minggu 192024 komunitas ...
97,"TIMESINDONESIA, SURABAYA – Memilih wewangian p...",TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,...",timesindonesia surabaya memilih wewangian peng...,timesindonesia surabaya pilih wewangian harum ...
98,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic...",timesindonesia magelang mencicipi kenikmatan s...,timesindonesia magelang cicip nikmat deret kul...


In [35]:
df.to_csv("/content/drive/My Drive/PPW/tugas/preprosesing-berita.csv",encoding='utf8', index=False)

## **TF-IDF**

TF-IDF (Term Frequency-Inverse Document Frequency) adalah metode yang digunakan untuk menentukan seberapa penting sebuah kata dalam suatu dokumen dibandingkan dengan dokumen-dokumen lainnya. Metode ini menggabungkan dua hal: *Term Frequency (TF), yaitu seberapa sering kata muncul di dalam sebuah dokumen, dan **Inverse Document Frequency (IDF)*, yang menunjukkan seberapa jarang kata tersebut muncul di seluruh kumpulan dokumen. Dengan begitu, kata yang sering muncul di dokumen tetapi jarang ada di dokumen lain dianggap lebih penting.

*Term Frequency* (TF) dihitung dengan melihat seberapa banyak kata muncul di dokumen, lalu dibandingkan dengan jumlah total kata di dokumen tersebut. Sementara itu, *Inverse Document Frequency* (IDF) dihitung berdasarkan seberapa jarang kata tersebut muncul di seluruh dokumen. Jika sebuah kata jarang muncul di dokumen lain, nilai IDF-nya tinggi, yang berarti kata itu lebih spesial atau memberikan informasi yang lebih bermakna dalam dokumen tersebut.

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Dimana:

$$
\text{TF}(t, d) = \frac{\text{Jumlah kemunculan } t \text{ dalam dokumen } d}{\text{Total jumlah kata dalam dokumen } d}
$$

$$
\text{IDF}(t, D) = \log\left(\frac{N}{|\{d \in D : t \in d\}|}\right)
$$

Ketika TF dan IDF dikalikan, hasilnya adalah bobot untuk setiap kata. Kata-kata yang sering muncul di dokumen tertentu tetapi jarang ada di dokumen lain akan mendapatkan bobot yang lebih tinggi, sehingga dianggap lebih penting.

In [36]:
df = pd.read_csv("/content/drive/My Drive/PPW/tugas/preprosesing-berita.csv")

In [37]:
x = df['stemming'].values
y = df['Kategori'].values

In [38]:
Xtrain, Xtest,Ytrain,Ytest = train_test_split(x,y,test_size=0.2,random_state=2)

In [40]:
vect = TfidfVectorizer()
X = vect.fit_transform(Xtrain)

X_array = vect.transform(Xtrain)

In [41]:
print(X_array.toarray())

[[0.         0.         0.         ... 0.         0.         0.08404587]
 [0.         0.         0.         ... 0.         0.         0.08404587]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [46]:
X_array.shape

(80, 1584)

In [43]:
# Menyimpan vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vect, f)

In [44]:
# Membuka file dan memuat vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vect = pickle.load(f)

# Menampilkan vectorizer
print(vect)

TfidfVectorizer()
